# Install all dependencies and setting up the environment

In [ ]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [ ]:
!pip install tensorflow==2.2.0

  Using cached tensorflow-2.2.0-cp37-cp37m-manylinux2010_x86_64.whl (516.2 MB)
  Using cached tensorflow_estimator-2.2.0-py2.py3-none-any.whl (454 kB)
  Using cached tensorboard-2.2.2-py3-none-any.whl (3.0 MB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.4.0
    Uninstalling tensorflow-estimator-2.4.0:
      Successfully uninstalled tensorflow-estimator-2.4.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.4.1
    Uninstalling tensorboard-2.4.1:
      Successfully uninstalled tensorboard-2.4.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.4.1
    Uninstalling tensorflow-2.4.1:
      Successfully uninstalled tensorflow-2.4.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tfx-bsl 0.29.0 requires tensorflow!=2.0.*,!=2.1.*,!=2.2.*,!=2.3.*,<

In [ ]:
import sys

# Confirm that we're using Python 3
assert sys.version_info.major is 3, 'Oops, not running Python 3. Use Runtime > Change runtime type'

In [ ]:
import tensorflow as tf

print('Installing TensorFlow Data Validation')
!pip install -q tensorflow_data_validation[visualization]

Installing TensorFlow Data Validation


In [ ]:
%pwd

'/content'

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


# You will get error in the following cell after insattling these dependencies so when u instal thes dependencies then restart your runtime and run all the cell again

In [ ]:
import pandas as pd

dataset = pd.read_csv("1.2 pollution_small.csv")

In [ ]:
!pip install apache-beam

In [ ]:
import tensorflow_data_validation as tfdv
print('TFDV version: {}'.format(tfdv.version.__version__))

TFDV version: 0.29.0


In [ ]:
dataset.shape

(2188, 5)

# Asigning the data in training 

In [ ]:
training_data = dataset[:1600]

# Describing some statistics about the training data set 

here the soot is the darget column 

In [ ]:
training_data.describe()

,pm10,no2,so2,soot
count,1600.000000,1600.000000,1600.000000,1600.000000
mean,49.656494,30.980519,16.229981,21.551956
std,35.211906,12.400788,10.621896,12.127354
min,6.380000,9.740000,4.010000,6.000000
25%,28.345000,22.567500,9.777500,14.400000
50%,38.835000,28.715000,13.275000,18.630000
75%,58.050000,36.370000,19.282500,24.072500
max,277.250000,138.010000,123.130000,107.650000


# Assigning the data for testing 

In [ ]:
test_data= dataset[1600:]

# describing the data 

In [ ]:
test_data.describe()

,pm10,no2,so2,soot
count,588.000000,588.000000,588.000000,588.000000
mean,44.648248,37.296922,13.605170,18.441310
std,28.992087,10.940050,5.098944,6.596459
min,11.900000,15.070000,4.990000,8.000000
25%,28.337500,29.217500,10.122500,14.410000
50%,35.555000,35.815000,12.345000,17.090000
75%,50.812500,43.872500,15.855000,20.962500
max,273.770000,106.030000,38.030000,87.210000


# data analysis and validation with tensorflow 

# generating training statistics 

In [ ]:
training_staictis = tfdv.generate_statistics_from_dataframe(dataframe=dataset)

In [ ]:
#training_staictis

# Infering Schema 

The schema codifies properties which the input data is expected to satisfy, such as data types or categorical values, and can be modified or replaced by the user.

In [ ]:
schema = tfdv.infer_schema(statistics=training_staictis)

#  Displaying the schema 

In [ ]:
tfdv.display_schema(schema)

,Type,Presence,Valency,Domain
Feature name,,,,
'Date',BYTES,required,,-
'pm10',FLOAT,required,,-
'no2',FLOAT,required,,-
'so2',FLOAT,required,,-
'soot',FLOAT,required,,-


# Calculate Test Statistics 

In [ ]:
test_statistic = tfdv.generate_statistics_from_dataframe(dataframe=test_data)

# Compare test statistics with the Schema

Checking for Annamolies 

# Anamaolies are the difference between the test data and schema 

In [ ]:
anomalies = tfdv.validate_statistics(statistics=test_statistic, schema=schema)

# Displaying all detected anomalies


In [ ]:
tfdv.display_anomalies(anomalies)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


#### Here we saw that no anomalies are found so for making anomalies we copy the dataset and the drop one column from that the do all the processes again the u will find out the anomolies  

# Coping the data 

In [ ]:
test_data_copy = test_data.copy()

# Now droping the one column for anomalies 

In [ ]:
test_data_copy.drop("soot" , axis=1 , inplace=True)

# again doing the staticts based on data with anomalies 

In [ ]:
test_data_copy_statistic = tfdv.generate_statistics_from_dataframe(dataframe=test_data_copy)

# New anomalies 

In [ ]:
anomalies_new = tfdv.validate_statistics(statistics=test_data_copy_statistic , schema=schema)

# Now displaying the anomoly 

In [ ]:
tfdv.display_anomalies(anomalies_new)

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)


,Anomaly short description,Anomaly long description
Feature name,,
'soot',Column dropped,Column is completely missing


 Here you can see that the anamolies are display 

# Now we preparing the schema for serving 

### WE are going to do the following for training and serving while during training the the drop column willnot be droping during training while in serving it would be consider drop 

creating serving enivroment and training enviroment 

In [ ]:
schema.default_environment.append("TRAINING")
schema.default_environment.append("SERVING")

# Removing the target column from serving schema 

In [ ]:
tfdv.get_feature(schema , "soot" ).not_in_environment.append("SERVING")

# Checking the anomolies for serving enviroments and new test set

In [ ]:
serving_env_anomalies = tfdv.validate_statistics(test_data_copy_statistic, schema , environment="SERVING")

In [ ]:
tfdv.display_anomalies(serving_env_anomalies )

/usr/local/lib/python3.7/dist-packages/tensorflow_data_validation/utils/display_util.py:188: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  pd.set_option('max_colwidth', -1)
